In [6]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd
from ta.volatility import AverageTrueRange
from datetime import datetime


def get_historical_data(stk):
    df = yf.download(stk, period='max', interval="1wk").dropna()
    return df

def trade_parameters(df):
    
    df['ATR']=AverageTrueRange(high=df.High,low=df.Low,close=df.Close,window=5,fillna=False).average_true_range().rank(pct=True)



    df=df.iloc[1:,::]

    return df


def get_atm_option_data(symbol,expiry_date):
    ticker = yf.Ticker(symbol)
    last_price = ticker.history(period=expiry_date).Close[0]
    calls = ticker.option_chain(date=expiry_date).calls
    puts = ticker.option_chain(date=expiry_date).puts
    atm_call = calls[calls.inTheMoney == True].iloc[-1, :]
    atm_put = puts[puts.inTheMoney == False].iloc[-1, :]
    strike = atm_put.strike
    atm_call_last = (atm_call.bid + atm_call.ask) / 2
    atm_put_last = (atm_put.bid + atm_put.ask) / 2
    atm_call_IV = atm_call.impliedVolatility
    atm_put_IV = atm_put.impliedVolatility
    atm_data = [
        symbol,
        last_price,
        strike,
        atm_call_last,
        atm_put_last,
        atm_call_IV,
        atm_put_IV,
    ]
    return atm_data


expiry_date='2022-09-02'
arr = []
atm_option_data = pd.read_csv("historical_straddle_data/option_data_2022-09-02_2022-08-29.csv")
tickers=atm_option_data.symbol
yields=(atm_option_data.atm_call_last+atm_option_data.atm_put_last)/atm_option_data.strike
for i,j in zip(tickers,yields):
    try:
        stock = i
        df = get_historical_data(stock)
        df = trade_parameters(df)
        df_opt=get_atm_option_data(stock,expiry_date)
        arr.append([i,
                    df.index[-2],## updated to reflect start of the past week
                    df.ATR[-2],
                    df_opt[2],
                    df_opt[3],
                    df_opt[4],
                    j/100
                   ]
                  )
    except: 
        pass
finaldf = pd.DataFrame(
data=arr,
columns=[
    "stock",
    'last date',
    'ATR',
    'strike',
    'call',
    'put',
    'target_yield'

],
)
finaldf['current_yield']=(finaldf.call+finaldf.put)/finaldf.strike
finaldf['yield_diff']=(finaldf.current_yield-finaldf.target_yield)/finaldf.target_yield
print(finaldf)
finaldf.to_csv('check.csv')

[*********************100%***********************]  1 of 1 completed


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********